**Đầu tiên ta chuẩn bị các hàm phụ trợ trong quá trình thực hiện mã hoá DES.**




In [1]:
# Chuyển đổi từ chuỗi thập lục phân sang nhị phân
def hexadecimal_to_binary(string):
    return bin(int(string, 16))[2:].zfill(len(string) * 4)

# Chuyển đổi từ chuỗi nhị phân sang thập lục phân
def binary_to_hexadecimal(string):
    return hex(int(string, 2))[2:].upper().zfill(len(string) // 4)

# Chuyển đổi từ chuỗi kí tự sang nhị phân
def string_to_binary(string):
    binary_result = ""

    for char in string:
        binary_char = bin(ord(char))[2:].zfill(8)
        binary_result += binary_char

    return binary_result

# Chuyển đổi từ số nhị phân sang chuỗi kí tự
def binary_to_string(binary):
    string_result = ""

    for i in range(0, len(binary), 8):
        binary_char = binary[i:i+8]
        decimal_char = int(binary_char, 2)
        string_result += chr(decimal_char)

    return string_result

# Chuyển đổi từ hệ thập phân sang nhị phân
def decimal_to_binary(number):
    bin_str = bin(number)[2:]
    return bin_str.zfill(((len(bin_str) + 3) // 4) * 4)

# Chuyển đổi từ hệ nhị phân sang thập phân
def binary_to_decimal(binary):
    decimal, i = 0, 0
    while binary != 0:
        dec = binary % 10
        decimal += dec * 2**i
        binary //= 10
        i += 1
    return decimal

# Hoán vị các ký tự trong chuỗi bits theo thứ tự được chỉ định bởi mảng arr
# Với n là độ dài của mảng
def permute(bits, arr):
    return ''.join(bits[i - 1] for i in arr)

# Dịch chuyển các ký tự trong chuỗi bits sang trái một số vị trí bằng n
# Các ký tự bị dịch ra khỏi đầu chuỗi sẽ được thêm vào cuối chuỗi
def shift_left(bits, n):
    return bits[n:] + bits[:n]

# Thực hiện phép toán XOR trên hai chuỗi nhị phân a và b
# Kết quả trả về là một chuỗi nhị phân mới
def xor(a, b):
    return ''.join('0' if bit1 == bit2 else '1' for bit1, bit2 in zip(a, b))

# Tạo ra một mảng mới ip_inv là mảng nghịch đảo của mảng ip đầu vào
def create_ip_inv(ip):
    ip_inv = [0] * len(ip)

    for index, value in enumerate(ip):
        ip_inv[value - 1] = index + 1

    return ip_inv

# Chuẩn hóa chuỗi lên 8 ký tự
def standardize_input(input):
    if len(input) < 8:
        # Thêm ký tự '_' vào cuối nếu thiếu
        return input.ljust(8, '_')
    elif len(input) > 8:
        # Cắt bớt nếu thừa
        return input[:8]
    else:
        # Giữ nguyên nếu đã đủ 8 ký tự
        return input

# Hàm in ra mảng theo hàng để dễ nhìn
def print_array_by_line(input_array, length_each_line):
    for i in range(0, len(input_array), length_each_line):
        print(input_array[i : i+length_each_line])

**Tiếp theo ta tạo các bảng dự liệu nhằm phục vụ cho giải thuật DES**

Đầu tiên ta phải tạo bảng IP (Initial Permutation) và IP_INV (Reverse Initial Permutation) nhằm hoán vị khối dữ liệu đầu vào điể đưa vào quá trình mã hoá và đặt chúng về vị trí cũ khi đã hoàn thành quá trình mã hoá. Chính vì vậy ta dựa vào IP để tạo bảng IP_INV

Tiếp theo đó là bảng E (Expansion) để mở rộng từ 32 lên 48 bit khi đầu vào đã hoán vị và được chia đôi

Ngoài ra còn có các bảng PC-1 (Permuted Choice 1), PC-2 (Permuted Choice 2) và  SHIFT_SCHEDULE (Bảng lưu số lần chuyển dịch) nhằm phục vụ cho 16 lần lặp

Tiếp đến là bảng S-boxes nhằm thay thế đầu vào 48 bits được chia đôi từ PLAIN TEXT và mở rộng từ E - một cách không tuyến tính bằng cách lấy 2 bit ĐẦU và 4 bit GIỮA của mỗi 8 khối dữ liệu (từ 48 bit đầu vào) (6bits * 8blocks = 48). Đầu ra sẽ là 8 khối dữ liệu, mỗi khối 4 bit cộng lại thành 32 bit

Cuối cùng là bảng hoán vị PBox nhằm áo trộn các bit sau khi chúng đã qua các S-boxes, tăng độ phức tạp và độ an toàn của kết quả mã hóa

In [2]:
# Tạo bảng IP mặc định
IP = [58, 50, 42, 34, 26, 18, 10, 2,
      60, 52, 44, 36, 28, 20, 12, 4,
      62, 54, 46, 38, 30, 22, 14, 6,
      64, 56, 48, 40, 32, 24, 16, 8,
      57, 49, 41, 33, 25, 17, 9, 1,
      59, 51, 43, 35, 27, 19, 11, 3,
      61, 53, 45, 37, 29, 21, 13, 5,
      63, 55, 47, 39, 31, 23, 15, 7]

# Tạo bảng IP_INV từ IP
IP_INV = create_ip_inv(IP)

# Tạo bảng E mặc định của DES để mở rộng 32 bit được chia ra thành 48 bit
# Bảng E (mở rộng R từ 32 bit lên 48 bit)
E = [32, 1, 2, 3, 4, 5,
     4, 5, 6, 7, 8, 9,
     8, 9, 10, 11, 12, 13,
     12, 13, 14, 15, 16, 17,
     16, 17, 18, 19, 20, 21,
     20, 21, 22, 23, 24, 25,
     24, 25, 26, 27, 28, 29,
     28, 29, 30, 31, 32, 1]

# Tạo các bảng PC1 - PC2 nhằm tạo ra 16 khoá còn từ khoá chính nhằm phục vụ cho 16 lần lặp
# Bảng PC-1 (Permuted Choice 1) - Hoán vị và nén khóa từ 64 bit xuống 56 bit
PC1 = [57, 49, 41, 33, 25, 17, 9,
       1, 58, 50, 42, 34, 26, 18,
       10, 2, 59, 51, 43, 35, 27,
       19, 11, 3, 60, 52, 44, 36,
       63, 55, 47, 39, 31, 23, 15,
       7, 62, 54, 46, 38, 30, 22,
       14, 6, 61, 53, 45, 37, 29,
       21, 13, 5, 28, 20, 12, 4]

# Bảng PC-2 (Permuted Choice 2) - Tạo khóa con 48 bit từ 56 bit
PC2 = [14, 17, 11, 24, 1, 5,
       3, 28, 15, 6, 21, 10,
       23, 19, 12, 4, 26, 8,
       16, 7, 27, 20, 13, 2,
       41, 52, 31, 37, 47, 55,
       30, 40, 51, 45, 33, 48,
       44, 49, 39, 56, 34, 53,
       46, 42, 50, 36, 29, 32]

# Số bit dịch trong mỗi vòng lặp
SHIFT_SCHEDULE = [1, 1, 2, 2, 2, 2, 2, 2,
                  1, 2, 2, 2, 2, 2, 2, 1]

# Tạo bảng S-boxes
S_BOXES = [
    [
        [14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
        [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
        [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
        [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13]
    ],
    [
        [15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
        [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
        [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
        [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9]
    ],
    [
        [10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
        [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
        [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
        [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12]
    ],
    [
        [7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
        [13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
        [10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
        [3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14]
    ],
    [
        [2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
        [14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
        [4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
        [11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3]
    ],
    [
        [12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11],
        [10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
        [9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
        [4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13]
    ],
    [
        [4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
        [13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
        [1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2],
        [6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12]
    ],
    [
        [13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7],
        [1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
        [7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
        [2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11]
    ]
]

# Tạo bảng hoán vị P (P-box)
P = [16, 7, 20, 21, 29, 12, 28, 17,
     1, 15, 23, 26, 5, 18, 31, 10,
     2, 8, 24, 14, 32, 27, 3, 9,
     19, 13, 30, 6, 22, 11, 4, 25]

print("Đã tạo các bảng dữ liệu phục vụ cho giải thuật DES")

Đã tạo các bảng dữ liệu phục vụ cho giải thuật DES


**Chuẩn bị 16 khoá con dựa vào khoá chính để phục vụ cho 16 lần lặp trong giải thuật DES**

In [3]:
key = standardize_input("12345678")

 # Chuẩn bị Subkeys
# Chuyển chuỗi khoá thành chuỗi nhị phân
key = string_to_binary(key)


# Đưa khoá về 56 bits bằng cách hoán vị với bảng PC1
key = permute(key, PC1)

# Chia khoá đá nén thành 56 bits thành 2 phần
left = key[0:28]
right = key[28:56]

binary_sub_keys = []
for i in range(0, 16):
    # Chuyển số bit của 2 phần đã tách ra từ khoá dựa trên bảng SHIFT_SCHEDULE
    left = shift_left(left, SHIFT_SCHEDULE[i])
    right = shift_left(right, SHIFT_SCHEDULE[i])

    # Sau đó kết hợp chugns lại với nhau
    combine_str = left + right

    # Và chuyển nó thành 48 bits dựa trên bảng PC2
    round_key = permute(combine_str, PC2)

    # Thêm vào danh sách khoá, tổng cộng có 16 khoá phục vụ cho 16 lần lặp
    binary_sub_keys.append(round_key)

# In ra kết quả
for i, subkey in enumerate(binary_sub_keys):
    print(f"\nSubkey {i + 1} là:")
    for j in range(0, len(subkey), 8):
        print(subkey[j:j+8])


Subkey 1 là:
01010000
00101100
10101100
01010111
00101010
11000010

Subkey 2 là:
01010000
10101100
10100100
01010000
10100011
01000111

Subkey 3 là:
11010000
10101100
00100110
11110110
10000100
10001100

Subkey 4 là:
11100000
10100110
00100110
01001000
00110111
11001011

Subkey 5 là:
11100000
10010110
00100110
00111110
11110000
00101001

Subkey 6 là:
11100000
10010010
01110010
01100010
01011101
01100010

Subkey 7 là:
10100100
11010010
01110010
10001100
10101001
00111010

Subkey 8 là:
10100110
01010011
01010010
11100101
01011110
01010000

Subkey 9 là:
00100110
01010011
01010011
11001011
10011010
01000000

Subkey 10 là:
00101111
01010001
01010001
11010000
11000111
00111100

Subkey 11 là:
00001111
01000001
11011001
00011001
00011110
10001100

Subkey 12 là:
00011111
01000001
10011001
11011000
01110000
10110001

Subkey 13 là:
00011111
00001001
10001001
00100011
01101010
00101101

Subkey 14 là:
00011011
00101000
10001101
10110010
00111001
10010010

Subkey 15 là:
00011001
00101100
10001100
1

**Cuối cùng là quá trình chính của giải thuật DES**

In [4]:
plain_text = standardize_input("12345678")

def DES_PROCESS(binary_input, binary_key):
    # Hoán vị đầu vào nhị phân với bảng hoán vị ban đầu IP
    binary_input = permute(binary_input, IP)

    # Sau khi hoán vị thị chia ra thành 2 đoạn, mỗi đoạn 32bits (64/2)
    L = binary_input[:32]
    R = binary_input[32:]

    for i in range(0, 16):
        #  Mở rộng khối cuối bằng cách hoán vị với bảng E từ 32bits lên 48bits
        R_expanded = permute(R, E)

        # Thực hiện phép xor với khoá con tương ứng đã tạo ở vòng lặp tương ứng
        # Vòng thứ i dùng khoá thứ i đã tạo ở bước trên
        XOR = xor(R_expanded, binary_key[i])

        # Qua các bước trên dữ liệu có độ dài 48 bits
        # Ta lấy chia làm 8 khối, mỗi khối 6 bits
        # Lấy 2 bits ở vị trí đầu và cuối cùng với 4 bits giữa để xác định vị trí của giá trị trong bảng S-boxes
        # Sau đó thay thế chúng với giá trị tương ứng trong bảng S-boxes
        sbox_str = ""
        for j in range(0, 8):
            # XOR[j * 6] là bit đầu tiên
            # XOR[j * 6 + 5] là bit cuối cùng
            # Kết hợp chúng để xác định hàng (row) trong bảng S-boxes
            row = binary_to_decimal(int(XOR[j * 6] + XOR[j * 6 + 5]))

            # Xác định 4 bits giữa để xác định cột (column) trong bảng S-boxes
            col = binary_to_decimal(int(XOR[j * 6 + 1] + XOR[j * 6 + 2] + XOR[j * 6 + 3] + XOR[j * 6 + 4]))
            # Thay thế giá trị (substitution) từ bảng S-boxes sử dụng hàng và cột đã xác định
            val = S_BOXES[j][row][col]

            # Thêm vào kết quả sau khi thay thế từ bảng S-boxex
            # mỗi lần thêm 4 bits trong bảng S-boxex, đầu ra sẽ là 8 * 4 = 32 bits
            sbox_str = sbox_str + decimal_to_binary(val)

        # Xáo trộn các bit lại một lần nữa bằng bảng P
        sbox_str = permute(sbox_str, P)

        # Thực hiện phép XOR giữa phần bên trái L và phần R đã thay thế bằng giá trị trong S-boxex
        result = xor(L, sbox_str)
        # Sau đó ta được giá trị mới của L
        L = result

        # Sau khi L và R đều đã thay đổi
        # Sau mỗi vòng lặp, nếu không phải là vòng lặp cuối cùng (i != 15), hoán đổi phần L và R
        if(i != 15):
            L, R = R, L

    # Sau khi kết thúc 16 lần lặp, ta kết hợp phần L và R đã thay đổi lại
    combine = L + R

    # Hoán vị sắp xếp lại các khối bits về vị trí ban đầu để tạo ra văn bản mã hóa
    cipher_text = permute(combine, IP_INV)

    return cipher_text


print("MÃ HOÁ")
print("Plain Text: ", plain_text)
# Để giải mã, ta đưa vào DES_PROCESS các khoá con đã tạo
cipher_text = DES_PROCESS(string_to_binary(plain_text), binary_sub_keys)
print("Cipher Text dưới dạng hexdecimal: ", binary_to_hexadecimal(cipher_text))

print("\nGIẢI MÃ")
# Ngược lại, để giải mã, ta đưa vào DES_PROCESS các khoá con theo chiều ngược lại
binary_sub_keys_reverse = binary_sub_keys[::-1]
text = binary_to_string(DES_PROCESS(cipher_text, binary_sub_keys_reverse))
print("Plain Text ban đầu sau khi giải mã: ", text)

MÃ HOÁ
Plain Text:  12345678
Cipher Text dưới dạng hexdecimal:  96D0028878D58C89

GIẢI MÃ
Plain Text ban đầu sau khi giải mã:  12345678


**Ở các phần trên mỗi lần chạy DES chỉ có thể thao tác trên một khối 8 kí tự, ở đây ta sẽ làm cho nó có thể mã hoá và giải mã trên đoạn dài hơn**

In [5]:
def encrypt_long_message(plain_text, binary_sub_keys):
    encrypted_message = ""

    # Chuyển đổi chuỗi plain_text thành dạng nhị phân
    binary_message = string_to_binary(plain_text)

    # Chia chuỗi nhị phân thành các khối 64 bits (8 ký tự)
    blocks = [binary_message[i:i + 64] for i in range(0, len(binary_message), 64)]

    for block in blocks:
        # Nếu khối có độ dài nhỏ hơn 64 bits, thêm '0' vào cuối để đảm bảo đủ 64 bits
        if len(block) < 64:
            block = block.ljust(64, '0')

        encrypted_block = DES_PROCESS(block, binary_sub_keys)
        encrypted_message += encrypted_block

    return encrypted_message

def decrypt_long_message(encrypted_message, binary_sub_keys):
    decrypted_message = ""

    # Chia chuỗi nhị phân đã mã hóa thành các khối 64 bits (8 ký tự)
    blocks = [encrypted_message[i:i + 64] for i in range(0, len(encrypted_message), 64)]

    for block in blocks:
        decrypted_block = DES_PROCESS(block, binary_sub_keys_reverse)
        decrypted_message += decrypted_block

    return decrypted_message

plain_text = "This is a longer text that needs encryption"

# Mã hóa
encrypted_message = encrypt_long_message(plain_text, binary_sub_keys)
print("Mã hoá:", binary_to_hexadecimal(encrypted_message))

# Giải mã
decrypted_message = decrypt_long_message(encrypted_message, binary_sub_keys_reverse)
print("Giải mã:", binary_to_string(decrypted_message))



Mã hoá: 05C9C4CAFB9937D994611F063379299C72B2752AC01410EAB65F11ECEB0B3909B5DF5DEDB4B46545CD4909F865CDEAFF
Giải mã: This is a longer text that needs encryption     
